# Listeners must have arguments "event" and "metadata" with proper types

In [1]:
from morpheus.common.types.event_sourcing.EventMetadata import EventMetadata
from morpheus.project.domain.events.ProjectEvents import ProjectCreatedEvent
from morpheus.common.infrastructure.event_sourcing.EventPublisher import EventPublisher, listen_to


class Invalid:
  pass


class ProjectorInvalid0:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self):
    print('Created project')


class ProjectorInvalid1:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, invalid: ProjectCreatedEvent, metadata: EventMetadata):
    print('Created project')


class ProjectorInvalid2:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: ProjectCreatedEvent, invalid: EventMetadata):
    print('Created project')


class ProjectorInvalid3:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: Invalid, metadata: EventMetadata):
    print('Created project')


class ProjectorInvalid4:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: ProjectCreatedEvent, metadata: Invalid):
    print('Created project')


event_publisher = EventPublisher()
try:
  projector = ProjectorInvalid0()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid1()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid2()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid3()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid4()
  event_publisher.register(projector)
except Exception as e:
  print(e)

Event listener <bound method ProjectorInvalid0.on_project_created of <__main__.ProjectorInvalid0 object at 0x138835b80>> must have an argument called "event" that should by type hinted with a subclass of EventBase
Event listener <bound method ProjectorInvalid1.on_project_created of <__main__.ProjectorInvalid1 object at 0x1395f2750>> must have an argument called "event" that should by type hinted with a subclass of EventBase
Event listener <bound method ProjectorInvalid2.on_project_created of <__main__.ProjectorInvalid2 object at 0x138835b80>> must have an argument called "metadata" that should by type hinted with a subclass of EventMetadata
Event listener <bound method ProjectorInvalid3.on_project_created of <__main__.ProjectorInvalid3 object at 0x11f715c70>> must have an argument called "event" that should by type hinted with a subclass of EventBase
Event listener <bound method ProjectorInvalid4.on_project_created of <__main__.ProjectorInvalid4 object at 0x138835b80>> must have an arg

# Cannot register same listener twice

In [2]:
class Projector:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: ProjectCreatedEvent, metadata: EventMetadata):
    print(f'Created project {event.get_project_id()} with metadata {metadata}')


try:
  event_publisher = EventPublisher()
  projector = Projector()
  event_publisher.register(projector)
  event_publisher.register(projector)
except Exception as e:
  print(e)

Event handler already registered


# Create projects

In [3]:
from morpheus.project.infrastructure.persistence.ProjectSummaryRepository import project_summary_repository
from morpheus.project.infrastructure.event_sourcing.ProjectEventStore import project_event_store
from morpheus.project.types.User import UserId
from morpheus.project.types.Project import ProjectId, Name, Description, Tags
from morpheus.project.application.write.ProjectCommandHandlers import CreateProjectCommand, CreateProjectCommandHandler

for i in range(10):
  create_project_command = CreateProjectCommand(
    project_id=ProjectId.new(),
    name=Name.from_str(f'Project {i}'),
    description=Description.from_str(f'Description {i}'),
    tags=Tags.from_list([f'tag{i}', f'tag{i + 1}']),
    created_by=UserId.new()
  )
  CreateProjectCommandHandler.handle(create_project_command)

event_envelopes = project_event_store.get_all_events_ordered_by_version()
print(f'Found {len(event_envelopes)} events')
for event_envelope in event_envelopes:
  print(f'"{event_envelope.get_event().get_event_name().to_str()}" occurred at {event_envelope.event.get_occurred_at().to_iso_with_timezone()}')

project_summaries = project_summary_repository.find_all()
print(f'Found {len(project_summaries)} project summaries')
for project_summary in project_summaries:
  print(f'Project {project_summary.project_id.to_str()}: {project_summary.project_name.to_str()}')

Found 30 events
"Project Created" occurred at 2024-01-29T13:06:11.760378+01:00
"Project Created" occurred at 2024-01-29T13:06:11.774077+01:00
"Project Created" occurred at 2024-01-29T13:06:11.781184+01:00
"Project Created" occurred at 2024-01-29T13:06:11.786952+01:00
"Project Created" occurred at 2024-01-29T13:06:11.792690+01:00
"Project Created" occurred at 2024-01-29T13:06:11.796189+01:00
"Project Created" occurred at 2024-01-29T13:06:11.799626+01:00
"Project Created" occurred at 2024-01-29T13:06:11.802020+01:00
"Project Created" occurred at 2024-01-29T13:06:11.804287+01:00
"Project Created" occurred at 2024-01-29T13:06:11.806481+01:00
"Project Created" occurred at 2024-01-29T13:07:45.317987+01:00
"Project Created" occurred at 2024-01-29T13:07:45.338184+01:00
"Project Created" occurred at 2024-01-29T13:07:45.345359+01:00
"Project Created" occurred at 2024-01-29T13:07:45.351762+01:00
"Project Created" occurred at 2024-01-29T13:07:45.371312+01:00
"Project Created" occurred at 2024-01-2

# Reproject project summaries

In [4]:
from morpheus.project.presentation.cli.ProjectionCliCommands import ReprojectProjectSummariesCliCommand

ReprojectProjectSummariesCliCommand.run()

Reprojecting project summaries
------------------------------

Found 30 events
Reset projector
Successfully reprojected project summaries


In [5]:
project_summaries = project_summary_repository.find_all()
print(f'Found {len(project_summaries)} project summaries')
for project_summary in project_summaries:
  print(f'Project {project_summary.project_id.to_str()}: {project_summary.project_name.to_str()}')

Found 30 project summaries
Project 6c8f560b-c153-403b-abda-f184fc8fde59: Project 0
Project 8707ffbf-ae97-4a71-bd8a-156f02c4caa3: Project 1
Project 12c0fa21-9254-462d-a764-b45662a597f2: Project 2
Project 325fd08f-29ad-409f-a249-e13897f72291: Project 3
Project 7b077c3e-2cbd-494f-bd12-28a9d9ac69f3: Project 4
Project 92d7f6b1-a67f-42d7-b686-a0ac90ef1eb0: Project 5
Project 40576811-ba44-491d-ba5a-d00548db0562: Project 6
Project 12dbdd18-1886-4015-a751-ce5af212a057: Project 7
Project e777f60a-9d3d-4546-8942-355c40b36114: Project 8
Project bcc9a6d4-36e3-4bbf-93fa-31435a4859c7: Project 9
Project ac232765-8aa4-4573-8bb5-be2d0f0de4a3: Project 0
Project 2076ba05-2586-4d73-897d-08ddffdb5c5e: Project 1
Project 7a1b6876-7edb-4047-aa61-7ef97b65e777: Project 2
Project 20d677a0-812e-474c-b53e-0e544715cb55: Project 3
Project a6fc8c24-9b84-4df2-8a63-921fd5b09c37: Project 4
Project 147bf39e-89be-4b61-8914-1fe02585107e: Project 5
Project 870909ef-beff-48ae-b1f4-0da5ead56a59: Project 6
Project a8421f6c-f9ce